In [9]:
#Import Generic Modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# For some Statistics
from scipy import stats
from scipy.stats import norm, skew
from sklearn.preprocessing import Imputer

# Standardizing numerical fefrom sklearn.metrics import classification_reportatures
from sklearn.preprocessing import StandardScaler

# Encoding categorical features
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

from sklearn import svm, datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import classification_report
# Label encoding for Agency type, distribution channel, claim, Gender 
from sklearn import preprocessing 
  
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, roc_curve


import os

In [10]:
# Creating path variable to read data
raw_data_path = os.path.abspath(os.path.join(os.path.join(os.path.join(os.path.join(os.getcwd(),os.path.pardir),'data'),'raw'),'train.csv'))
root_path = os.path.abspath(os.path.join(os.path.join(os.getcwd(),os.path.pardir),os.path.pardir))
# Creating path variable to read data
raw_data_path_test = os.path.abspath(os.path.join(os.path.join(os.path.join(os.path.join(os.getcwd(),os.path.pardir),'data'),'raw'),'test.csv'))

In [11]:
df = pd.read_csv(raw_data_path)
df['Gender'] = df['Gender'].fillna(method='backfill')
df['Gender'] = df['Gender'].fillna(df['Gender'].mode()[0])

In [12]:
df_test = pd.read_csv(raw_data_path_test)
df_test['Gender'] = df_test['Gender'].fillna(method='backfill')
df_test['Gender'] = df_test['Gender'].fillna(df_test['Gender'].mode()[0])

In [13]:
# Encode labels in column 'Agency Type'. 
df_test['Agency Type']= label_encoder.fit_transform(df_test['Agency Type']) 
df_test['Agency Type'].unique()
df['Agency Type']= label_encoder.fit_transform(df['Agency Type']) 
df['Agency Type'].unique()


# Encode labels in column Distribution Channel
df_test['Distribution Channel']= label_encoder.fit_transform(df_test['Distribution Channel']) 
df_test['Distribution Channel'].unique()
df['Distribution Channel']= label_encoder.fit_transform(df['Distribution Channel']) 
df['Distribution Channel'].unique()

array([1, 0], dtype=int64)

In [14]:
df.Gender.isnull().sum()

0

In [15]:
# Treating outliers in duration column

df[df['Duration']<0]
# replacing negative duration by median
df["Duration"] = np.where(df["Duration"] <0, df.Duration.median(),df['Duration'])

# Treating outliers in Age column
df[df['Age']==0].Age.count()
df1 = df[df['Age']!=0]
df1 = df1[df1['Age']<=100]
median_val= df1.Age.median()
# replacing age == 0 by median
df["Age"] = np.where(df["Age"] ==0, median_val,df['Age'])
df[df['Age']>=100].Age.count()
# replacing values >100 duration by median
df["Age"] = np.where(df["Age"] >100, median_val,df['Age'])

df.head()

,ID,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Gender,Age
0,3433,CWT,1,1,Rental Vehicle Excess Insurance,0,7.0,MALAYSIA,0.0,17.82,F,31.0
1,4339,EPX,1,1,Cancellation Plan,0,85.0,SINGAPORE,69.0,0.00,F,36.0
2,34590,CWT,1,1,Rental Vehicle Excess Insurance,0,11.0,MALAYSIA,19.8,11.88,F,75.0
3,55816,EPX,1,1,2 way Comprehensive Plan,0,16.0,INDONESIA,20.0,0.00,F,32.0
4,13816,EPX,1,1,Cancellation Plan,0,10.0,"KOREA, REPUBLIC OF",15.0,0.00,F,29.0


In [16]:
# replacing negative duration by median
df_test["Duration"] = np.where(df_test["Duration"] <0, df_test.Duration.median(),df_test['Duration'])

# replacing values in Age>100 duration by median
df2 = df_test[df_test['Age']!=0]
df2 = df2[df2['Age']<=100]
median_val_1= df2.Age.median()

df_test["Age"] = np.where(df_test["Age"] >100, median_val_1,df_test['Age'])
                                                                    
# replacing age == 0 by median
df_test["Age"] = np.where(df_test["Age"] ==0, median_val_1,df_test['Age'])                                                                    

In [17]:
df.head()

,ID,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Gender,Age
0,3433,CWT,1,1,Rental Vehicle Excess Insurance,0,7.0,MALAYSIA,0.0,17.82,F,31.0
1,4339,EPX,1,1,Cancellation Plan,0,85.0,SINGAPORE,69.0,0.00,F,36.0
2,34590,CWT,1,1,Rental Vehicle Excess Insurance,0,11.0,MALAYSIA,19.8,11.88,F,75.0
3,55816,EPX,1,1,2 way Comprehensive Plan,0,16.0,INDONESIA,20.0,0.00,F,32.0
4,13816,EPX,1,1,Cancellation Plan,0,10.0,"KOREA, REPUBLIC OF",15.0,0.00,F,29.0


In [18]:
#Creating bins for categorical features (Destination)

df2 = pd.DataFrame(df_test.groupby('Destination')['Net Sales'].count()).reset_index()#-500-810
df2.rename(columns={'Net Sales':'Claim_count'},inplace=True)
df1 = pd.merge(df_test,df2, on = 'Destination')
df_test['country_bin'] = df1['Claim_count'].apply(lambda x: '> 500' if x > 500 else
                                                                     'Between 0 and 250'  if ((x>0) and (x < 250)) else
                                                                     'Between 250 and 500'   if ((x>250) and (x < 500)) else
                                                                     'Less than -250'  )

# encoding for df
df2 = pd.DataFrame(df.groupby('Destination')['Net Sales'].count()).reset_index()
df2.rename(columns={'Net Sales':'Claim_count'},inplace=True)
df1 = pd.merge(df,df2, on = 'Destination')
df['country_bin'] = df1['Claim_count'].apply(lambda x: '> 500' if x > 500 else
                                                                     'Between 0 and 250'  if ((x>0) and (x < 250)) else
                                                                     'Between 250 and 500'   if ((x>250) and (x < 500)) else
                                                                     'Less than -250'  )

In [19]:
df_test.head()

,ID,Agency,Agency Type,Distribution Channel,Product Name,Duration,Destination,Net Sales,Commision (in value),Gender,Age,country_bin
0,25902,CWT,1,1,Rental Vehicle Excess Insurance,58.0,ICELAND,49.5,29.7,F,47.0,Between 0 and 250
1,19535,EPX,1,1,2 way Comprehensive Plan,4.0,SINGAPORE,20.0,0.0,F,24.0,Between 0 and 250
2,43717,EPX,1,1,2 way Comprehensive Plan,68.0,VIET NAM,20.0,0.0,F,36.0,Between 0 and 250
3,48271,EPX,1,1,2 way Comprehensive Plan,41.0,CHINA,28.0,0.0,F,36.0,Between 0 and 250
4,52318,EPX,1,1,Cancellation Plan,16.0,SINGAPORE,11.0,0.0,F,36.0,Between 0 and 250


In [20]:
#Creating bins for categorical features (Agency)
df2 = pd.DataFrame(df_test.groupby('Agency')['Net Sales'].count()).reset_index()
df2.rename(columns={'Net Sales':'Claim_count'},inplace=True)
df1 = pd.merge(df_test,df2, on = 'Agency')
df_test['agency_bin'] = df1['Claim_count'].apply(lambda x: '> 500' if x > 500 else
                                                                     'Between 0 and 250'  if ((x>0) and (x < 250)) else
                                                                     'Between 250 and 500'   if ((x>250) and (x < 500)) else
                                                                     'Less than -250'  )

# encoding fo df
df2 = pd.DataFrame(df.groupby('Agency')['Net Sales'].count()).reset_index()
df2.rename(columns={'Net Sales':'Claim_count'},inplace=True)
df1 = pd.merge(df,df2, on = 'Agency')
df['agency_bin'] = df1['Claim_count'].apply(lambda x: '> 500' if x > 500 else
                                                                     'Between 0 and 250'  if ((x>0) and (x < 250)) else
                                                                     'Between 250 and 500'   if ((x>250) and (x < 500)) else
                                                                     'Less than -250'  )

In [21]:
#Creating bins for categorical features (Product Name)
df2 = pd.DataFrame(df_test.groupby('Product Name')['Net Sales'].count()).reset_index()
df2.rename(columns={'Net Sales':'Claim_count'},inplace=True)
df1 = pd.merge(df_test,df2, on = 'Product Name')
df_test['product_bin'] = df1['Claim_count'].apply(lambda x: '> 500' if x > 500 else
                                                                     'Between 0 and 250'  if ((x>0) and (x < 250)) else
                                                                     'Between 250 and 500'   if ((x>250) and (x < 500)) else
                                                                     'Less than -250'  )

# encoding for df
df2 = pd.DataFrame(df.groupby('Product Name')['Net Sales'].count()).reset_index()
df2.rename(columns={'Net Sales':'Claim_count'},inplace=True)
df1 = pd.merge(df,df2, on = 'Product Name')
df['product_bin'] = df1['Claim_count'].apply(lambda x: '> 500' if x > 500 else
                                                                     'Between 0 and 250'  if ((x>0) and (x < 250)) else
                                                                     'Between 250 and 500'   if ((x>250) and (x < 500)) else
                                                                     'Less than -250'  )


In [22]:

df_test=df_test[['agency_bin','product_bin','country_bin','Agency Type','Distribution Channel','Duration','Net Sales','Commision (in value)','Age','Gender']]
df=df[['agency_bin','product_bin','country_bin','Agency Type','Distribution Channel','Claim','Duration','Net Sales','Commision (in value)','Age','Gender']]

In [23]:
ag_dummies = pd.get_dummies(df_test.agency_bin)
ag_dummies.columns = ['agency_> 500','agency_Between 0 and 250']
df_test = pd.concat([df_test, ag_dummies], axis=1)
# encoding for df
ag_dummies = pd.get_dummies(df.agency_bin)
ag_dummies.columns = ['agency_> 500','agency_Between 0 and 250','agency_Between 250 and 500']
df = pd.concat([df, ag_dummies], axis=1)

In [24]:
ag_dummies = pd.get_dummies(df_test.product_bin)
ag_dummies.columns = ['product_> 500','product_Between 0 and 250','product_Between 250 and 500']
df_test = pd.concat([df_test, ag_dummies], axis=1)
# encoding for df
ag_dummies = pd.get_dummies(df.product_bin)
ag_dummies.columns = ['product_> 500','product_Between 0 and 250','product_Between 250 and 500']
df = pd.concat([df, ag_dummies], axis=1)

In [25]:
ag_dummies = pd.get_dummies(df_test.country_bin)
ag_dummies.columns = ['country_> 500','country_Between 0 and 250','country_Between 250 and 500']
df_test = pd.concat([df_test, ag_dummies], axis=1)
# encoding for df
ag_dummies = pd.get_dummies(df.country_bin)
ag_dummies.columns = ['country_> 500','country_Between 0 and 250','country_Between 250 and 500']
df = pd.concat([df, ag_dummies], axis=1)

In [26]:
df_test.drop(columns = ['agency_bin','product_bin','country_bin'],axis=1,inplace=True)
df.drop(columns = ['agency_bin','product_bin','country_bin'],axis=1,inplace=True)

In [27]:
df_test['Gender']= label_encoder.fit_transform(df_test['Gender'])
df['Gender']= label_encoder.fit_transform(df['Gender'])

In [28]:
df_test.head()

,Agency Type,Distribution Channel,Duration,Net Sales,Commision (in value),Age,Gender,agency_> 500,agency_Between 0 and 250,product_> 500,product_Between 0 and 250,product_Between 250 and 500,country_> 500,country_Between 0 and 250,country_Between 250 and 500
0,1,1,58.0,49.5,29.7,47.0,0,1,0,1,0,0,0,1,0
1,1,1,4.0,20.0,0.0,24.0,0,1,0,1,0,0,0,1,0
2,1,1,68.0,20.0,0.0,36.0,0,1,0,1,0,0,0,1,0
3,1,1,41.0,28.0,0.0,36.0,0,1,0,1,0,0,0,1,0
4,1,1,16.0,11.0,0.0,36.0,0,1,0,1,0,0,0,1,0


In [29]:
df_test['agency_Between 250 and 500'] = 0

df_test = df_test[['Agency Type', 'Distribution Channel', 'Duration', 'Net Sales',
       'Commision (in value)', 'Age', 'Gender', 'agency_> 500',
       'agency_Between 0 and 250', 'agency_Between 250 and 500',
       'product_> 500', 'product_Between 0 and 250',
       'product_Between 250 and 500', 'country_> 500',
       'country_Between 0 and 250', 'country_Between 250 and 500']]

### Resample data with Random undersampling

In [30]:
# Split entire dataset into train test data

#X = trans.drop('Class', axis=1)
X = df
y = df['Claim']

X_train_random_undersamp, X_test_random_undersamp, y_train_random_undrsamp, y_test_random_undersamp = train_test_split(X, y, train_size=0.7, random_state=7)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [31]:
X_train_random_undersamp.shape

(35462, 17)

In [32]:
X_train_random_undersamp.Claim.value_counts()

0    34959
1      503
Name: Claim, dtype: int64

In [33]:
X_test_random_undersamp.shape

(15198, 17)

In [34]:
X_test_random_undersamp.Claim.value_counts()

0    14959
1      239
Name: Claim, dtype: int64

In [35]:
y_train_random_undrsamp.shape

(35462,)

In [36]:
y_test_random_undersamp.shape

(15198,)

In [37]:
#Find Number of samples which are minority
minority_Claim = len(X_train_random_undersamp[X_train_random_undersamp['Claim'] == 1])
print(np.array(minority_Claim))

#Get indices of undersampled data
majority_indices = X_train_random_undersamp[X_train_random_undersamp.Claim == 0].index
print(majority_indices)

#Random sample with minority indices
random_indices = np.random.choice(majority_indices,minority_Claim,replace=False)
#print(random_indices)

#Find the indices of majority samples
majority_indices = X_train_random_undersamp[X_train_random_undersamp.Claim == 1].index
#print(majority_indices)

#Concat fraud indices with sample non-fraud ones
under_sample_indices = np.concatenate([majority_indices,random_indices])
#print(under_sample_indices)

#Get Balance Dataframe
df_final_random_undersample = X_train_random_undersamp.loc[under_sample_indices]
y_ROV = df_final_random_undersample['Claim']
df_final_random_undersample.drop('Claim', axis=1 ,inplace=True)


503
Int64Index([ 9199,  4150, 26328, 14412, 44367,  9144,  6971, 37781, 41597,
            40815,
            ...
            19623, 45960, 27354, 28232,  2583, 13927,   919, 38467, 10742,
            49689],
           dtype='int64', length=34959)


In [38]:
df_final_random_undersample.head()
y_ROV.head()

19598    1
25535    1
26488    1
39996    1
45552    1
Name: Claim, dtype: int64

In [39]:
df_final_random_undersample.shape

(1006, 16)

In [40]:
y_test_random_undersamp.shape

(15198,)

### Applying Random Forest Classifier on Randomly Undersampled data

In [41]:
X_test_random_undersamp.drop(['Claim'], axis=1, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [42]:
X_test_random_undersamp.shape

(15198, 16)

In [43]:
df_test.head()

,Agency Type,Distribution Channel,Duration,Net Sales,Commision (in value),Age,Gender,agency_> 500,agency_Between 0 and 250,agency_Between 250 and 500,product_> 500,product_Between 0 and 250,product_Between 250 and 500,country_> 500,country_Between 0 and 250,country_Between 250 and 500
0,1,1,58.0,49.5,29.7,47.0,0,1,0,0,1,0,0,0,1,0
1,1,1,4.0,20.0,0.0,24.0,0,1,0,0,1,0,0,0,1,0
2,1,1,68.0,20.0,0.0,36.0,0,1,0,0,1,0,0,0,1,0
3,1,1,41.0,28.0,0.0,36.0,0,1,0,0,1,0,0,0,1,0
4,1,1,16.0,11.0,0.0,36.0,0,1,0,0,1,0,0,0,1,0


In [44]:

clf = RandomForestClassifier(n_jobs=2, random_state=0)
clf.fit(df_final_random_undersample,y_ROV)
y_predict = clf.predict(df_test)
    

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [45]:
y_predict=pd.DataFrame(y_predict)

In [47]:
df_test_new = pd.read_csv(raw_data_path_test)

In [48]:
df_test_new = df_test_new['ID']
df_test_new = df_test_new.reset_index()

In [49]:
df_test_new.head()

,index,ID
0,0,25902
1,1,19535
2,2,43717
3,3,48271
4,4,52318


In [55]:
y_predict = clf.predict(X_test_random_undersamp)
df5 = pd.DataFrame(y_predict_real)
df5.reset_index(inplace =True)
df6 = pd.merge(df5,df_test_new, on ='index')
df6.drop('index',axis=1,inplace=True)
df6.columns=['Claim','ID']
df6 = df6[['ID','Claim']]

In [56]:
df6.head()

,ID,Claim
0,25902,0
1,19535,0
2,43717,0
3,48271,0
4,52318,0


In [57]:
df6.to_csv("submission.csv")

In [58]:
pd.crosstab(pd.Series(y_predict), pd.Series(y_ROV))


Claim,0,1
row_0,,
0,105,93
1,47,39


In [59]:
print(classification_report(y_predict,y_test_random_undersamp))


              precision    recall  f1-score   support

           0       0.71      0.99      0.83     10676
           1       0.67      0.04      0.07      4522

   micro avg       0.71      0.71      0.71     15198
   macro avg       0.69      0.51      0.45     15198
weighted avg       0.70      0.71      0.60     15198



In [204]:
generate_model_report(y_test_random_undersamp, y_predict)


Accuracy =  0.7461508093170154
Precision =  0.0387458577619169
Recall =  0.6359832635983264
F1 score =  0.07304180682364249


### Resampling using SMOTE OverSampling

In [208]:
X = df
y = df['Claim']
  
X_train_SMOTE, X_test_SMOTE, y_train_SMOTE, y_test_SMOTE = train_test_split(X, y, train_size=0.7, random_state=7)

print(len(X_train_SMOTE))
print(len(y_train_SMOTE))
print(len(X_test_SMOTE))
print(len(y_test_SMOTE))


35462
35462
15198
15198


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [209]:
X_train_SMOTE.head()

,Agency Type,Distribution Channel,Claim,Duration,Net Sales,Commision (in value),Age,Gender,agency_> 500,agency_Between 0 and 250,agency_Between 250 and 500,product_> 500,product_Between 0 and 250,product_Between 250 and 500,country_> 500,country_Between 0 and 250,country_Between 250 and 500
9199,1,1,0,37.0,23.00,0.00,36.0,1,1,0,0,1,0,0,1,0,0
4150,0,1,0,104.0,35.00,12.25,38.0,1,1,0,0,1,0,0,1,0,0
26328,0,1,0,6.0,18.00,6.30,58.0,1,1,0,0,1,0,0,1,0,0
14412,1,1,0,12.0,19.00,0.00,36.0,0,1,0,0,1,0,0,1,0,0
44367,0,1,0,13.0,1.13,0.32,48.0,1,1,0,0,1,0,0,1,0,0


In [210]:

smote = SMOTE(random_state=2)
X_ov, y_ov = smote.fit_resample(X_train_SMOTE, y_train_SMOTE)

X_ov = pd.DataFrame(X_ov) 
pd.DataFrame(X_ov).columns = df.columns
print(len(X_ov))
print(len(y_ov))

69918
69918


In [211]:
X_ov.Claim.value_counts()

1.0    34959
0.0    34959
Name: Claim, dtype: int64

In [215]:
X_ov.drop(['Claim'], axis=1, inplace=True)

### Applying SVM Classifier Classification Technique

In [ ]:
    model = svm.SVC(kernel='linear')
    model.fit(X_ov, y_ov)
    plt.plot(kind='SVC')

### Applying Random Forest Classifier

In [216]:
clf = RandomForestClassifier(n_jobs=2, random_state=0)

In [217]:
clf.fit(X_ov,y_ov)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=2,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [219]:
X_test_SMOTE.drop(['Claim'], axis=1,inplace=True )

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [220]:
y_predict = clf.predict(X_test_SMOTE)

In [221]:
pd.crosstab(pd.Series(y_predict), pd.Series(y_test_SMOTE))

Claim,0,1
row_0,,
0,4417,68
1,57,1


In [222]:
print(classification_report(y_predict,y_test_SMOTE))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99     15012
           1       0.04      0.05      0.04       186

   micro avg       0.97      0.97      0.97     15198
   macro avg       0.51      0.52      0.51     15198
weighted avg       0.98      0.97      0.97     15198



In [223]:
generate_model_report(y_test_SMOTE, y_predict)

Accuracy =  0.9732201605474404
Precision =  0.04838709677419355
Recall =  0.03765690376569038
F1 score =  0.04235294117647059


### Applying Decision Tree Classifier

In [60]:
dtc = DecisionTreeClassifier()

In [62]:
dtc.fit(df_final_random_undersample,y_ROV)
#y_predict_dtc = dtc.predict(X_test_SMOTE)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [63]:
y_predict = dtc.predict(X_test_random_undersamp)
df5 = pd.DataFrame(y_predict_real)
df5.reset_index(inplace =True)
df6 = pd.merge(df5,df_test_new, on ='index')
df6.drop('index',axis=1,inplace=True)
df6.columns=['Claim','ID']
df6 = df6[['ID','Claim']]

In [64]:
df6.to_csv("submission_21.csv")

In [65]:
print(classification_report(y_predict,y_test_random_undersamp))


              precision    recall  f1-score   support

           0       0.64      0.99      0.78      9695
           1       0.67      0.03      0.06      5503

   micro avg       0.64      0.64      0.64     15198
   macro avg       0.66      0.51      0.42     15198
weighted avg       0.65      0.64      0.52     15198



In [66]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score

precision_score(y_test_random_undersamp, y_predict)


0.02907504997274214

In [226]:
pd.crosstab(pd.Series(y_predict_dtc), pd.Series(y_test_SMOTE))

Claim,0,1
row_0,,
0,4370,68
1,104,1


In [227]:
print(classification_report(y_predict_dtc,y_test_SMOTE))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98     14833
           1       0.08      0.05      0.06       365

   micro avg       0.96      0.96      0.96     15198
   macro avg       0.53      0.52      0.52     15198
weighted avg       0.96      0.96      0.96     15198



In [172]:
def generate_model_report(y_actual, y_predicted):
    print("Accuracy = ", accuracy_score(y_actual, y_predicted))
    print("Precision = ", precision_score(y_actual, y_predicted))
    print("Recall = ", recall_score(y_actual, y_predicted))
    print("F1 score = ", f1_score(y_actual, y_predicted))

In [206]:
def generate_auc_roc_curve(clf, X_test):
    y_pred_proba = clf.predict_proba(X_test)[:,1]
    fpr, tpr, thresholds = roc_curve(X_test, y_pred_proba)
    auc = roc_auc_score(X_test, y_pred_proba)
    plt.plot(fpr, tpr, label="AUC ROC Curve with Area Under the curve =" + str(auc))
    plt.legend(loc=4)
    plt.show()
    pass